### Importando Librerías

In [1]:
from typing import TypedDict, Annotated, List
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace, HuggingFaceEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage, BaseMessage
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langchain.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv
from huggingface_hub import login
from langchain.prompts import ChatPromptTemplate
from typing import TypedDict, List, Union, Sequence
from langgraph.prebuilt import ToolNode

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma

/Users/user/anaconda3/envs/env-rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
load_dotenv()
login(os.environ["HUGGINGFACE_API_TOKEN"])

### Rag en Español - Solvex, Funcional.

In [7]:
import json
from typing import TypedDict, Annotated, List, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace, HuggingFaceEmbeddings, HuggingFaceEndpointEmbeddings
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
import os
from dotenv import load_dotenv
from huggingface_hub import login

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_chroma import Chroma

In [8]:
DIRECTORIO_PERSISTENTE = "db_chroma"
PATH_A_PRODUCTOS = "productos_pdf"

print("--- Iniciando el proceso de indexación de archivos PDF ---")

# --- 2. Cargar los Documentos PDF ---

# Comprobamos si la carpeta de productos existe.
if not os.path.exists(PATH_A_PRODUCTOS):
    raise FileNotFoundError(f"La carpeta '{PATH_A_PRODUCTOS}' no fue encontrada. "
                             "Por favor, créala y coloca tus archivos PDF dentro.")

# Usamos DirectoryLoader para cargar todos los archivos .pdf de la carpeta.
# Le especificamos que use PyPDFLoader para cada archivo.
loader = DirectoryLoader(
    PATH_A_PRODUCTOS,
    glob="**/*.pdf",       # Patrón para encontrar todos los archivos .pdf
    loader_cls=PyPDFLoader
)

documents = loader.load()

if not documents:
    raise ValueError(f"No se encontraron documentos PDF en la carpeta '{PATH_A_PRODUCTOS}'. "
                     "Asegúrate de que la ruta es correcta y que los archivos están ahí.")

print(f"Se han cargado {len(documents)} páginas desde los archivos PDF en la carpeta '{PATH_A_PRODUCTOS}'.")

--- Iniciando el proceso de indexación de archivos PDF ---
Se han cargado 15 páginas desde los archivos PDF en la carpeta 'productos_pdf'.


In [ ]:
load_dotenv()

True

: 

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B")

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Tamaño de cada trozo en caracteres
    chunk_overlap=200   # Caracteres de superposición para mantener el contexto
)

chunks = text_splitter.split_documents(documents)

print(f"Los documentos se han dividido en {len(chunks)} trozos (chunks).")

print("Creando el Vector Store con ChromaDB. Esto puede tardar unos minutos...")
# El resto del proceso es idéntico. ChromaDB toma los chunks de texto extraídos de los PDFs.
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=DIRECTORIO_PERSISTENTE,
    collection_name="productos"
)
print("Vector Store creado con éxito!")

Los documentos se han dividido en 50 trozos (chunks).
Creando el Vector Store con ChromaDB. Esto puede tardar unos minutos...
Vector Store creado con éxito!


In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
hf_endpoint = HuggingFaceEndpoint(
    repo_id=model_id,
    task="text-generation",
    temperature=0.2, # A lower temperature can make it follow JSON instructions better
    huggingfacehub_api_token=userdata.get('HUGGINGFACE_API_TOKEN'),
    max_new_tokens=1024,
)
llm = ChatHuggingFace(llm=hf_endpoint)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# --- 3. Tool Definition (sin cambios) ---
@tool
def retriever_tool(query: str) -> str:
  """
  Busca y devuelve información relevante de los documentos de productos de Solvex.
  """
  print(f"--- Ejecutando Retriever con la consulta: '{query}' ---")
  docs = retriever.invoke(query)
  if not docs:
    return "No se encontró información en los documentos para esta consulta."
  return "\n\n".join([doc.page_content for doc in docs])


tools = [retriever_tool]
tools_dict = {t.name: t for t in tools}

# --- 3. Definición del Grafo del Agente ---
class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], add_messages]

def call_llm(state: AgentState):
    print("--- Llamando al LLM ---")
    messages = state['messages']
    response = llm.invoke(messages)
    print(f"--- Respuesta en crudo del LLM:\n{response.content}\n---")
    return {"messages": [response]}

def take_action(state: AgentState):
    print("--- Tomando Acción ---")
    last_message = state['messages'][-1].content
    try:
        action_data = json.loads(last_message)
        tool_name = action_data.get("tool_name")
        query = action_data.get("query")

        if tool_name in tools_dict:
            result = tools_dict[tool_name].invoke(query)
            tool_message = ToolMessage(content=str(result), name=tool_name, tool_call_id="manual_call")
            return {"messages": [tool_message]}
        else:
            error_message = AIMessage(content=f"Error: La herramienta '{tool_name}' no fue encontrada.")
            return {"messages": [error_message]}
    except json.JSONDecodeError:
        # Si la respuesta no es JSON, es la respuesta final del agente.
        # En este punto, no hacemos nada, ya que el borde `should_continue` la llevará al final.
        return

def should_continue(state: AgentState):
    last_message_content = state['messages'][-1].content
    try:
        # Una forma robusta de verificar si es un JSON con las claves correctas
        data = json.loads(last_message_content)
        if "tool_name" in data and "query" in data:
            return "action"
    except json.JSONDecodeError:
        pass # No es un JSON, por lo tanto, es la respuesta final.

    return END

graph = StateGraph(AgentState)
graph.add_node("llm", call_llm)
graph.add_node("action", take_action)
graph.set_entry_point("llm")
graph.add_conditional_edges("llm", should_continue, {"action": "action", END: END})
graph.add_edge("action", "llm")
rag_agent = graph.compile()

# --- 4. Lógica de Ejecución del Agente ---
def running_agent():
  print("\n--- SolvexBot INICIALIZADO (Modo de Consulta de Productos) ---")

  # --- CAMBIO 2: System Prompt mejorado y en español ---
  system_prompt = """
  Eres SolvexBot, un asistente de inteligencia artificial experto en los productos y servicios de la empresa Solvex. Tu única fuente de conocimiento son los documentos de producto que te han sido proporcionados.

  Tu tarea es responder a las preguntas de los usuarios de manera profesional, amable y precisa, simulando una conversación de WhatsApp.

  **Instrucciones de Operación:**
  1.  Cuando un usuario te haga una pregunta sobre un producto, primero debes usar la herramienta 'retriever_tool' para buscar información en los documentos.
  2.  Para usar la herramienta, DEBES responder ÚNICAMENTE con un objeto JSON en el siguiente formato:
      {"tool_name": "retriever_tool", "query": "una pregunta concisa que resuma la duda del usuario"}
  3.  Una vez que recibas la información del 'retriever_tool' (el CONTEXTO), debes formular una respuesta final para el usuario.

  **Reglas para Responder:**
  -   **BASA TU RESPUESTA ESTRICTAMENTE EN EL CONTEXTO PROPORCIONADO.** No utilices ningún conocimiento externo.
  -   Si el contexto no contiene la información necesaria para responder la pregunta, DEBES decir amablemente: "Lo siento, no he podido encontrar esa información específica en los documentos de nuestros productos." NO INVENTES RESPUESTAS.
  -   Si la pregunta del usuario es un saludo o no está relacionada con los productos (ej. "¿cómo estás?"), responde de manera cordial y breve sin usar la herramienta.
  -   Habla siempre en español.
  """
  initial_messages = [SystemMessage(content=system_prompt)]

  while True:
    user_input = input("\n👤 Tu pregunta (o escribe 'salir' para terminar): ")
    if user_input.lower() in ["salir", "exit", "quit"]:
      print("🤖 ¡Hasta luego! Gracias por consultar con SolvexBot.")
      break

    current_state = {"messages": initial_messages + [HumanMessage(content=user_input)]}
    result = rag_agent.invoke(current_state)

    print("\n🤖 SolvexBot:")
    print(result['messages'][-1].content)

In [ ]:
running_agent()


--- SolvexBot INICIALIZADO (Modo de Consulta de Productos) ---

👤 Tu pregunta (o escribe 'salir' para terminar): Si un gerente olvida firmar un documento importante, ¿cómo ayuda la aplicación?
--- Llamando al LLM ---
--- Respuesta en crudo del LLM:
{"tool_name": "retriever_tool", "query": "¿cómo ayuda la aplicación si un gerente olvida firmar un documento importante?"}
---
--- Tomando Acción ---
--- Ejecutando Retriever con la consulta: '¿cómo ayuda la aplicación si un gerente olvida firmar un documento importante?' ---
--- Llamando al LLM ---
--- Respuesta en crudo del LLM:
Lo siento, pero parece que hay un problema con mi respuesta anterior. Como resultado, no he podido comprender la pregunta del usuario.
Lo sentimos, no hemos podido encontrar información relevante sobre cómo la aplicación ayuda a un gerente que olvida firmar un documento importante.
---

🤖 SolvexBot:
Lo siento, pero parece que hay un problema con mi respuesta anterior. Como resultado, no he podido comprender la pre

KeyboardInterrupt: Interrupted by user

## Pruebas Unitarias

In [2]:
import requests
import json

# URL del endpoint de tu API corriendo localmente
api_url = "http://127.0.0.1:8000/query"

headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}

# Datos de la consulta que queremos enviar
data = {
    "user_id": "user-test-001",
    "query": "¿Qué problema principal busca resolver ProjectFlow PM en las organizaciones?"
}

print(f"Enviando petición POST a: {api_url}")
print(f"Datos: {json.dumps(data, indent=2)}")

try:
    # Realizar la petición POST
    response = requests.post(api_url, headers=headers, data=json.dumps(data))
    
    # Verificar que la petición fue exitosa (código 200)
    response.raise_for_status() 
    
    print("\n--- ¡Respuesta recibida! ---")
    print(json.dumps(response.json(), indent=2, ensure_ascii=False))

except requests.exceptions.RequestException as e:
    print(f"\n--- Error al conectar con la API ---")
    print(f"Detalle del error: {e}")

Enviando petición POST a: http://127.0.0.1:8000/query
Datos: {
  "user_id": "user-test-001",
  "query": "\u00bfQu\u00e9 problema principal busca resolver ProjectFlow PM en las organizaciones?"
}

--- ¡Respuesta recibida! ---
{
  "user_id": "user-test-001",
  "response": "El proyecto ProjectFlow PM busca resolver el problema principal de la falta de visibilidad y estandarización en la gestión de proyectos en las organizaciones. La herramienta proporciona una plataforma centralizada para la planificación, asignación de recursos, seguimiento de tiempos, gestión de riesgos y generación de informes de estado de todos los proyectos, facilitando la toma de decisiones proactiva y la entrega exitosa de los proyectos."
}


In [ ]:
# Imprimir de manera linda
print(response.json()["response"])

ProjectFlow PM busca resolver el problema principal de la falta de visibilidad y estandarización en la gestión de proyectos en las organizaciones. Proporciona una plataforma centralizada que se integra con el ecosistema de Microsoft, facilitando la planificación, asignación de recursos, seguimiento de tiempos, gestión de riesgos y generación de informes de estado en un solo lugar.


##### Promp 1

In [ ]:
data = {
    "user_id": "user-test-001",
    "query": "¿Cuál es el objetivo principal de la aplicación FormeSX?"
}

try:
    # Realizar la petición POST
    response = requests.post(api_url, headers=headers, data=json.dumps(data))
    
    # Verificar que la petición fue exitosa (código 200)
    response.raise_for_status() 
    
    print("\n--- ¡Respuesta recibida! ---")
    print(json.dumps(response.json(), indent=2, ensure_ascii=False))

except requests.exceptions.RequestException as e:
    print(f"\n--- Error al conectar con la API ---")
    print(f"Detalle del error: {e}")

##### Promp 2

In [ ]:
data = {
    "user_id": "user-test-001",
    "query": "¿Qué significa que el servicio CloudFlow MSP es proactivo en lugar de reactivo?"
}

try:
    # Realizar la petición POST
    response = requests.post(api_url, headers=headers, data=json.dumps(data))
    
    # Verificar que la petición fue exitosa (código 200)
    response.raise_for_status() 
    
    print("\n--- ¡Respuesta recibida! ---")
    print(json.dumps(response.json(), indent=2, ensure_ascii=False))

except requests.exceptions.RequestException as e:
    print(f"\n--- Error al conectar con la API ---")
    print(f"Detalle del error: {e}")

##### Promp 3

In [ ]:
data = {
    "user_id": "user-test-001",
    "query": "Cuales son los beneficios clave de cloud flow MSP?"
}

try:
    # Realizar la petición POST
    response = requests.post(api_url, headers=headers, data=json.dumps(data))
    
    # Verificar que la petición fue exitosa (código 200)
    response.raise_for_status() 
    
    print("\n--- ¡Respuesta recibida! ---")
    print(json.dumps(response.json(), indent=2, ensure_ascii=False))

except requests.exceptions.RequestException as e:
    print(f"\n--- Error al conectar con la API ---")
    print(f"Detalle del error: {e}")

##### Promp 4

In [ ]:
data = {
    "user_id": "user-test-001",
    "query": "¿Qué es Media Shelter?"
}

try:
    # Realizar la petición POST
    response = requests.post(api_url, headers=headers, data=json.dumps(data))
    
    # Verificar que la petición fue exitosa (código 200)
    response.raise_for_status() 
    
    print("\n--- ¡Respuesta recibida! ---")
    print(json.dumps(response.json(), indent=2, ensure_ascii=False))

except requests.exceptions.RequestException as e:
    print(f"\n--- Error al conectar con la API ---")
    print(f"Detalle del error: {e}")

##### Promp 5

In [ ]:
data = {
    "user_id": "user-test-001",
    "query": "¿Ofrecen plataformas para capacitación?"
}

try:
    # Realizar la petición POST
    response = requests.post(api_url, headers=headers, data=json.dumps(data))
    
    # Verificar que la petición fue exitosa (código 200)
    response.raise_for_status() 
    
    print("\n--- ¡Respuesta recibida! ---")
    print(json.dumps(response.json(), indent=2, ensure_ascii=False))

except requests.exceptions.RequestException as e:
    print(f"\n--- Error al conectar con la API ---")
    print(f"Detalle del error: {e}")

##### Promp 6

In [ ]:
data = {
    "user_id": "user-test-001",
    "query": "Qué soluciones ofrecen de Inteligencia de Negocios e Inteligencia Artificial?"
}

try:
    # Realizar la petición POST
    response = requests.post(api_url, headers=headers, data=json.dumps(data))
    
    # Verificar que la petición fue exitosa (código 200)
    response.raise_for_status() 
    
    print("\n--- ¡Respuesta recibida! ---")
    print(json.dumps(response.json(), indent=2, ensure_ascii=False))

except requests.exceptions.RequestException as e:
    print(f"\n--- Error al conectar con la API ---")
    print(f"Detalle del error: {e}")

##### Promp 7

In [ ]:
data = {
    "user_id": "user-test-001",
    "query": "¿Qué soluciones Tecnológicas Ofrecen Ustedes?"
}

try:
    # Realizar la petición POST
    response = requests.post(api_url, headers=headers, data=json.dumps(data))
    
    # Verificar que la petición fue exitosa (código 200)
    response.raise_for_status() 
    
    print("\n--- ¡Respuesta recibida! ---")
    print(json.dumps(response.json(), indent=2, ensure_ascii=False))

except requests.exceptions.RequestException as e:
    print(f"\n--- Error al conectar con la API ---")
    print(f"Detalle del error: {e}")

##### Promp 8

In [ ]:
data = {
    "user_id": "user-test-001",
    "query": "¿Ustedes hacen CRMs?, somos una agencia de viajes, y necesitamos de su ayuda."
}

try:
    # Realizar la petición POST
    response = requests.post(api_url, headers=headers, data=json.dumps(data))
    
    # Verificar que la petición fue exitosa (código 200)
    response.raise_for_status() 
    
    print("\n--- ¡Respuesta recibida! ---")
    print(json.dumps(response.json(), indent=2, ensure_ascii=False))

except requests.exceptions.RequestException as e:
    print(f"\n--- Error al conectar con la API ---")
    print(f"Detalle del error: {e}")